In [1]:
import sqlite3
import pandas as pd
import json
import secrets
import numpy as np
import tools
import os
import constants
import plotly.express as px

In [2]:
DB_NAME = constants.DB_NAME
SECRETS_FILE = constants.SECRETS_FILE

In [7]:
def generate_predictions(filename):
    data = pd.read_csv(constants.REF_DATA_PATH)
    data["Yield"] += np.random.normal(0, 0.1, data.shape[0])    
    data.to_csv(filename, index=False, header=False)
    return data
generate_predictions("test.txt")

,Yield
0,58.047740
1,66.493559
2,66.199424
3,64.508943
4,63.303488
...,...
1667,70.383242
1668,69.355399
1669,78.060428
1670,82.606301


## Open the DB  

In [26]:
def generate_secret_file(filename,groups):
    data= "[passwords]\n"
    for group_id, password in zip(groups["group_id"],groups["password"]):
        group_id = group_id.replace(" ","_")
        data += f"{group_id} = \"{password}\"\n"
    with open(filename,"w") as f:
        f.write(data)

def generate_group_table(filename):
    with open(filename) as f:
        data = json.load(f)
    for group_id in data.keys():
        for key,value in data[group_id].items():
            if type(value) == list:
                data[group_id][key] = ','.join(value)
    groups = [{"group_id":key, **value ,"password":secrets.token_urlsafe(16)} for key,value in data.items()]
    if not os.path.exists(".streamlit"):
        os.makedirs(".streamlit")
    groups = pd.DataFrame(groups)
    generate_secret_file(SECRETS_FILE,pd.DataFrame(groups))
    groups.drop(columns=["password"],inplace=True)
    return groups

def generat_dummy_scores(group_table, n=10):
    scores = []
    for group_id in group_table["group_id"]:
        for i in range(n):
            scores.append({
                "group_id":group_id,
                "score":np.random.random()*100,
                "date":pd.Timestamp.now()
            })
    scores = pd.DataFrame(scores)
    return scores

def create_scrore_table(db_name,group_table,n=10):
    conn = sqlite3.connect(db_name)
    if n>0:
        scores = generat_dummy_scores(group_table,n)
        scores.to_sql("score",conn,if_exists="replace",index=False)
    else:
        cur = conn.cursor()
        cur.execute("CREATE TABLE score (group_id TEXT, score REAL, date TEXT)")
        conn.commit()
    conn.close()


In [29]:


os.remove(DB_NAME)

if os.path.exists(DB_NAME):
    db_infos = tools.read_db(DB_NAME)
else:
    print("Creating new database")
    group_table = generate_group_table('groups.json')
    group_table.to_sql('group',sqlite3.connect(DB_NAME),index=False)
    create_scrore_table(DB_NAME,group_table,0)
    db_infos = tools.read_db(DB_NAME)


Creating new database


In [30]:
scores = db_infos["score"]
scores

,group_id,score,date


In [18]:
scores.groupby("group_id", as_index=False).min().sort_values("score", ascending=True)

,group_id,score,date
1,group 2,14.461390,2024-03-22 12:18:54.012658
0,group 1,29.018391,2024-03-22 12:18:54.012360


In [41]:
def get_leaderboard(db_filename):
    db_infos = tools.read_db(db_filename)
    scores = db_infos["score"].groupby("group_id", as_index=False).min().sort_values("score", ascending=True).reset_index(drop=True)
    scores["date"] = pd.to_datetime(scores["date"])
    scores["date"] = scores["date"].dt.strftime('%Y-%m-%d %H:%M:%S')
    scores["names"] = db_infos["group"].set_index("group_id").loc[scores["group_id"]]["names"].apply(lambda x: x.replace(",", ", ")).values
    #scores = scores.drop("group_id", axis=1)
    scores.loc[0,"group_id"] = "👑 "+ scores.loc[0,"group_id"]
    scores.index = scores.index + 1
    return scores[["group_id","names", "score", "date"]]
get_leaderboard(DB_NAME)

,group_id,names,score,date
1,👑 group 2,"KASMI Boutayna, ZERIATE Rim",14.461390,2024-03-22 12:18:54
2,group 1,"Mehdi ALLANI, Rayane HAMAIDI",29.018391,2024-03-22 12:18:54


In [45]:
scores

,group_id,score,date
0,group 1,98.360343,2024-03-22 12:18:54
1,group 1,99.405332,2024-03-22 12:18:54
2,group 1,76.313697,2024-03-22 12:18:54
3,group 1,69.079126,2024-03-22 12:18:54
4,group 1,91.921431,2024-03-22 12:18:54
5,group 1,71.997889,2024-03-22 12:18:54
6,group 1,91.515681,2024-03-22 12:18:54
7,group 1,29.018391,2024-03-22 12:18:54
8,group 1,68.734166,2024-03-22 12:18:54
9,group 1,69.465118,2024-03-22 12:18:54


In [44]:
group_id = "group 2"
px.line(scores[scores["group_id"]==group_id], x="date", y="score", title=f"Score evolution for {group_id}")

In [6]:
groups = db_infos["group"]

In [8]:
groups

,group_id,names,emails,password
0,group 1,"Mehdi ALLANI,Rayane HAMAIDI","Mehdi.ALLANI@etu.uca.fr,Rayane.HAMAIDI@etu.uca.fr",OPQ51fMmHR_JkhW4R6S2bg
1,group 2,"KASMI Boutayna,ZERIATE Rim","Boutayna.KASMI@etu.uca.fr,Rim.ZERIATE@etu.uca.fr",wsZFBcXoGf0jR1mTF2Va9A
